In [ ]:
import ifcopenshell
from rdflib import Graph, Namespace, URIRef, Literal, RDF
from py2neo import Graph as Neo4jGraph, Node, Relationship
import re

# === CONFIGURAÇÕES ===
IFC_FILE = "C:\\Users\\selah\\Downloads\\Building-Architecture.ifc"  # substitua pelo caminho do seu arquivo IFC
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "test-ifc"
NEO4J_DATABASE = "ifctest"

print("🔄 Iniciando o processo de importação do modelo IFC para o Neo4j...")

# === CONECTAR AO BANCO ===
graph_db = Neo4jGraph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# === CARREGAR O MODELO IFC ===
model = ifcopenshell.open(IFC_FILE)
produtos = model.by_type("IfcProduct")
print(f"✅ Modelo carregado com {len(produtos)} produtos IFC.")

# === CRIAR GRAFO RDF COM DADOS BÁSICOS ===
ns = Namespace("http://example.org/ifc/")
rdf_graph = Graph()

for element in produtos:
    if not element.GlobalId:
        continue

    uri = URIRef(ns[element.GlobalId])
    rdf_graph.add((uri, RDF.type, URIRef(ns[element.is_a()])))
    if element.Name:
        rdf_graph.add((uri, ns["name"], Literal(element.Name)))

print(f"📊 Grafo RDF gerado com {len(rdf_graph)} triplas.")

# === ENVIAR PARA O NEO4J (BANCO 'ifc-test') ===
for s, p, o in rdf_graph:
    subj = Node("IFCElement", uri=str(s))
    pred = re.sub(r'\W|^(?=\d)', '_', str(p).split("/")[-1])

    if isinstance(o, URIRef):
        obj = Node("IFCElement", uri=str(o))
        graph_db.run(f"USE {NEO4J_DATABASE} "
                     f"MERGE (a:IFCElement {{uri: $s}}) "
                     f"MERGE (b:IFCElement {{uri: $o}}) "
                     f"MERGE (a)-[r:{pred}]->(b)", s=str(s), o=str(o))
    else:
        graph_db.run(f"USE {NEO4J_DATABASE} "
                     f"MERGE (a:IFCElement {{uri: $s}}) "
                     f"SET a.{pred} = $o", s=str(s), o=str(o))

print("✅ Grafo carregado no banco `ifc-test` com sucesso!")

🔄 Iniciando o processo de importação do modelo IFC para o Neo4j...
✅ Modelo carregado com 22 produtos IFC.
📊 Grafo RDF gerado com 44 triplas.


ClientError: [Statement.SyntaxError] Invalid input '-': expected a database name, '(', 'FOREACH', '.', 'ALTER', 'ORDER BY', 'CALL', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNION', 'UNWIND', 'USE', 'WITH' or <EOF> (line 1, column 8 (offset: 7))
"USE ifc-test MERGE (a:IFCElement {uri: $s}) MERGE (b:IFCElement {uri: $o}) MERGE (a)-[r:_22_rdf_syntax_ns_type]->(b)"
        ^